In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o-2024-08-06",
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.R1,
    },
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

/Users/kevinlights/app/miniforge3/envs/open_manus/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:608: UserWarning: Could not find <think>..</think> field in model response content. No thought was extracted.
  thought, content = parse_r1_content(content)


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=127), metadata={}, content="Leaves whisper to each other, rustling in air,\nAs seasons turn and summer's heat does disappear.\nCrisp breezes dance with scarlet, orange flair,\nWhile pumpkins rest, under cool autumn stare.\n\nApples blush beneath morning frosty light,\nAnd trees shed their vibrant cloaks of green.\nA sweater worn; a cozy cup of malt\nBrings warmth back to the night and day.\n\nSilence falls, amidst nature's soft decay,\nEach leaf that drifts in twilight's silver play,\nIs but a gentle reminder of time's slow sway,\nWelcome fall, with all your hues and ways.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=43), metadata={}, content='Your poem beautifully captures t

In [3]:
await team.reset()  # Reset the team for a new task.
result = await team.run(task="写一首关于春天的短诗")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='写一首关于春天的短诗', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=31), metadata={}, content='春风轻拂绿意浓，\n桃红柳絮舞微风。\n莺歌燕语花间闹，\n一曲生机满园中。', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=59), metadata={}, content='这首诗描绘了春天生机勃勃的景象，语言流畅，意境优美。建议在最后一句可以稍微调整一下表达，比如改为“一曲春歌唱不休”，这样既呼应开头的春风，又增加了一点动态感。其他地方都很好！APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")


In [4]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=29, completion_tokens=126) metadata={} content="Leaves of amber and gold, they swirl and dance,\nIn the crisp air of autumn's gentle stance.\nWhispering secrets to chilly breezes high,\nAs summer fades and winter draws nigh.\n\nBursts of orange and red in twilight scenes,\nWhere once green was king, now color dreams.\nNature puts on her finest, a glorious show,\nWith every leaf, fall paints a brand-new glow.\n\nPumpkins carved with glee, bats roost in trees,\nCorn mazes winding under sapphire skies.\nHues of nature's palette come to life,\nIn the season that we fondly prize." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=166, completion_tokens=214) metadata={} content="Your poem captures the essence and beauty of autumn vividly. To enhance it further, consider adding more specific imagery

In [5]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves whisper in the cool breeze,
Crimson, gold, and amber please.
Sunsets paint the fading light,
As autumn breathes its crisp delight.

Pumpkins glow beneath the moon's gaze,
And jack-o'-lanterns welcome days
Of stories told around the fire,
While leaves crunch under wandering feet desire.
---------- critic ----------
Your poem captures the essence of fall beautifully with vivid imagery and a cozy atmosphere. Here are some minor suggestions to enhance it further:
- Consider adding a line about the changing temperature or a gusty wind to highlight the coolness of autumn.
- You might add an alliteration in "sunsets" to "sullen skies" for a more cohesive rhythm.

Here's how you could integrate these ideas:

Leaves whisper in the cool, crisp breeze,
Crimson, gold, and amber please.
Sullen skies with fading light,
As autumn breathes its chill delight.

Pumpkins glow beneath the moon’s gaze,

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=69), metadata={}, content="Leaves whisper in the cool breeze,\nCrimson, gold, and amber please.\nSunsets paint the fading light,\nAs autumn breathes its crisp delight.\n\nPumpkins glow beneath the moon's gaze,\nAnd jack-o'-lanterns welcome days\nOf stories told around the fire,\nWhile leaves crunch under wandering feet desire.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=109, completion_tokens=155), metadata={}, content='Your poem captures the essence of fall beautifully with vivid imagery and a cozy atmosphere. Here are some minor suggestions to enhance it further:\n- Consider adding a line about the changing temperature or a gusty wind to highlight the coolness of autumn.\n- You might add an alli

In [6]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
# Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.
external_termination.set()

# Wait for the team to finish.
await run

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Autumn whispers in the breeze,
Leaves of amber and gold release.
Crisp air dances through the trees,
 painting the ground in hues that please.

Pumpkins carved, Halloween's cheer,
Apples plucked from orchards near.
Warmsthebones, a cozy fire,
As twilight falls with gentlepire.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=67), metadata={}, content="Autumn whispers in the breeze,\nLeaves of amber and gold release.\nCrisp air dances through the trees,\n painting the ground in hues that please.\n\nPumpkins carved, Halloween's cheer,\nApples plucked from orchards near.\nWarmsthebones, a cozy fire,\nAs twilight falls with gentlepire.", type='TextMessage')], stop_reason='External termination requested')

In [7]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- critic ----------
Your poem is filled with warmth and autumnal imagery. Here are some minor tweaks to refine it further:

Autumn whispers in the breeze,
Leaves of amber and gold release.
Crisp air dances through the trees,
painting the ground in hues that please.

Pumpkins carved, Halloween's cheer,
Apples plucked from orchards near.
Warm the bones, a cozy fire,
As twilight falls with gentle fire.

APPROVE


TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=348, completion_tokens=91), metadata={}, content="Your poem is filled with warmth and autumnal imagery. Here are some minor tweaks to refine it further:\n\nAutumn whispers in the breeze,\nLeaves of amber and gold release.\nCrisp air dances through the trees,\npainting the ground in hues that please.\n\nPumpkins carved, Halloween's cheer,\nApples plucked from orchards near.\nWarm the bones, a cozy fire,\nAs twilight falls with gentle fire.\n\nAPPROVE", type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [8]:
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- user ----------
将这首诗用中文唐诗风格写一遍。
---------- primary ----------
当然可以，以下是按照唐代诗歌风格改写的版本：

秋风轻语意萧疏，
金黄琥珀叶纷飘。
寒气轻舞林间径，
染得大地彩妆乔。

南瓜雕刻庆万圣，
苹果丰收近田园。
炉火温暖入骨髓，
晚霞渐落映柔辉。
---------- critic ----------
您的翻译非常优美，很好地保留了原诗的意境和唐代诗歌风格。以下是稍作调整以确保更加顺畅和符合传统唐诗特点的版本：

秋风轻语意萧疏，
金黄琥珀叶纷飘。
寒气舞动林间径，
染得大地彩妆娇。

南瓜雕刻庆 Halloween，
苹果丰收近果园旁。
炉火温暖入骨髓，
晚霞渐落映柔辉。

请注意，“Halloween”和“果园旁”在唐诗中可能不太自然，如果能用中文词汇替换可能会更好。以下是调整后的版本：

秋风轻语意萧疏，
金黄琥珀叶纷飘。
寒气舞动林间径，
染得大地彩妆娇。

南瓜雕刻庆佳节，
苹果丰收近园旁。
炉火温暖入骨髓，
晚霞渐落映柔辉。

APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='将这首诗用中文唐诗风格写一遍。', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=294, completion_tokens=70), metadata={}, content='当然可以，以下是按照唐代诗歌风格改写的版本：\n\n秋风轻语意萧疏，\n金黄琥珀叶纷飘。\n寒气轻舞林间径，\n染得大地彩妆乔。\n\n南瓜雕刻庆万圣，\n苹果丰收近田园。\n炉火温暖入骨髓，\n晚霞渐落映柔辉。', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=528, completion_tokens=185), metadata={}, content='您的翻译非常优美，很好地保留了原诗的意境和唐代诗歌风格。以下是稍作调整以确保更加顺畅和符合传统唐诗特点的版本：\n\n秋风轻语意萧疏，\n金黄琥珀叶纷飘。\n寒气舞动林间径，\n染得大地彩妆娇。\n\n南瓜雕刻庆 Halloween，\n苹果丰收近果园旁。\n炉火温暖入骨髓，\n晚霞渐落映柔辉。\n\n请注意，“Halloween”和“果园旁”在唐诗中可能不太自然，如果能用中文词汇替换可能会更好。以下是调整后的版本：\n\n秋风轻语意萧疏，\n金黄琥珀叶纷飘。\n寒气舞动林间径，\n染得大地彩妆娇。\n\n南瓜雕刻庆佳节，\n苹果丰收近园旁。\n炉火温暖入骨髓，\n晚霞渐落映柔辉。\n\nAPPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [9]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
# Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
#     # Disable parallel tool calls for this example.
#     parallel_tool_calls=False,  # type: ignore
# )

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
    parallel_tool_calls=False,  # type: ignore
)


# Create a tool for incrementing a number.
def increment_number(number: int) -> int:
    """Increment a number by 1."""
    return number + 1


# Create a tool agent that uses the increment_number function.
looped_assistant = AssistantAgent(
    "looped_assistant",
    model_client=model_client,
    tools=[increment_number],  # Register the tool.
    system_message="You are a helpful AI assistant, use the tool to increment the number until it reach to the target value.",
)

# Termination condition that stops the task if the agent responds with a text message.
termination_condition = TextMessageTermination("looped_assistant")

# Create a team with the looped assistant agent and the termination condition.
team = RoundRobinGroupChat(
    [looped_assistant],
    termination_condition=termination_condition,
)

# Run the team with a task and print the messages to the console.
async for message in team.run_stream(task="Increment the number 5 to 10."):  # type: ignore
    print(type(message).__name__, message)

TextMessage source='user' models_usage=None metadata={} content='Increment the number 5 to 10.' type='TextMessage'
ToolCallRequestEvent source='looped_assistant' models_usage=RequestUsage(prompt_tokens=170, completion_tokens=125) metadata={} content=[FunctionCall(id='call_t1ugekza', arguments='{"number":5}', name='increment_number'), FunctionCall(id='call_hs7e8wo3', arguments='{"number":6}', name='increment_number'), FunctionCall(id='call_kvcznbkc', arguments='{"number":8}', name='increment_number'), FunctionCall(id='call_liwqg17o', arguments='{"number":9}', name='increment_number')] type='ToolCallRequestEvent'
ToolCallExecutionEvent source='looped_assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='6', name='increment_number', call_id='call_t1ugekza', is_error=False), FunctionExecutionResult(content='7', name='increment_number', call_id='call_hs7e8wo3', is_error=False), FunctionExecutionResult(content='9', name='increment_number', call_id='call_kvcznbkc'